<h1>Inteligência Aritifical - Algoritmo Genético - Problemas das 8 Rainhas <h1>

<h3> Aluno: Brainer Sueverti de Campos 

Ra: 790829 <h3>

<h3> 
Bibliotecas necessárias:

random - para gerar números aleatórios
pandas - para realizar as análises
<h3>

In [14]:
import random

<h3> Primeiramente serão definidas todas as funções necessárias para a solução do problema. <h3>

<h3> A função abaixo gera os cromossomos aleatóriamente utilizando a função random. Cada posição do vetor (cromossomo) está a posição da rainha na coluna.<h3>

In [15]:
def cromossomos_aleatorios(n):
    return [random.randint(0, n - 1) for _ in range(n)]

In [16]:
def pop_aletoria():
    x = []
    for i in range(0,pop_max):
        x.append(cromossomos_aleatorios(num_queen))
    return x

<h3> A função fitness abaixo retorna a quantidade de pares de rainhas que não estão em colisão. Para o caso n=8, o número ideal de fitness é n*(n-1)/2 = 28. Primeiramente é calculada as colisões horizontais e depois as colisões diagonais, que subtrai do fitness ideal.<h3>

In [17]:
def fitness(cromossomo):

    #fitness ideal -> 28
    max_fit = (len(cromossomo) * (len(cromossomo) - 1)) / 2


    #Cálculo das colisões horizontais
    colisoes_horizontais = (
        sum([cromossomo.count(queen) - 1 for queen in cromossomo]) / 2
    )

    #Cálculo das colisões diagonais
    n = len(cromossomo)
    esq_diagonal = [0] * (2 * n - 1)
    dir_diagonal = [0] * (2 * n - 1)
    for i in range(n):
        esq_diagonal[i + cromossomo[i] - 1] += 1
        dir_diagonal[len(cromossomo) - i + cromossomo[i] - 2] += 1

    colisoes_diagonais = 0
    for i in range(2 * n - 1):
        counter = 0
        if esq_diagonal[i] > 1:
            counter += esq_diagonal[i] - 1
        if dir_diagonal[i] > 1:
            counter += dir_diagonal[i] - 1
        colisoes_diagonais += counter

    return int(max_fit - (colisoes_horizontais + colisoes_diagonais))

In [18]:
def all_fitness(populacao):
  
  fit = []
  for i in range(len(populacao)):
    f = fitness(populacao[i])
    fit.append(f)

  return fit

<h3> 
A função abaixo realiza o cruzamento entre dois cromossomos, utilizando como resultado a primeira metade de um cromossomo x como sua primeira metade e a segunda metade como a segunda metade de um cromossomo y.

Exemplo:  

x = [1,2,3,4,1,2,3,4]

y = [5,6,7,8,5,6,7,8]

resultado = [1,2,3,4,5,6,7,8]
<h3>

In [19]:
def crossover(x,y):
    #tamanho do cromossomo
    n = len(x)
    #criação do resultado do crossover
    resultado1 = [0] * n
    resultado2 = [0] * n
    #crossover
    #a primeira metade do resultado é a primeira metade de x
    #a segunda metade do resultado é a segunda metade de y
    for i in range(n):
        if i<int(n/2):
            resultado1[i] = x[i]
            resultado2[i] = y[i]
        else:
            resultado1[i] = y[i]
            resultado2[i] = x[i]
    return resultado1,resultado2

<h3> função abaxio realiza a mutação em determinada taxa, na qual é selecionada uma posição e mudado o valor. <h3>

In [20]:
def mutacao(filhos,taxa):

  for i in range(len(filhos)):
    if random.random() < taxa:
      pos = random.randint(0,num_queen-1)
      filhos[i][pos] = random.randint(0,num_queen-1)

  return filhos

<h3> A seleção dos cromossomos que serão cruzados se dá através da função abaixo. É sorteado dois números aleatórios como posição do vetor fitnesses, que armazenam os valores da função fitness para cada cromossomo da população, utilizando o valor da posição é possível encontrar os cromossomos e, assim, dentre esses dois cromossomos é feito a escolha do que tiver o maior valor da função fitness, isso é feito para selecionar dois cromossomos. <h3>  

In [21]:
def selecao_torneio(fitnesses):
  
  ind1 = -1
  ind2 = -1

  while ind1 == ind2:
    #primeiro selecionado
    sorteados = random.sample(range(0,pop_max), 2)
    if fitnesses[sorteados[0]] > fitnesses[sorteados[1]]:
      ind1 = sorteados[0]
    else:
      ind1 = sorteados[1]
    #segundo selecionado
    sorteados = random.sample(range(0,pop_max), 2)
    if fitnesses[sorteados[0]] > fitnesses[sorteados[1]]:
      ind2 = sorteados[0]
    else:
      ind2 = sorteados[1]  
        
  return ind1,ind2

<h3> A função abaixo seleciona os dois maiores fitness da população <h3>

In [22]:
def elitismo(fitnesses):

  x = fitnesses.index(max(fitnesses))
  fitnesses[x] = 0
  y = fitnesses.index(max(fitnesses))

  return x,y

<h3>Abaixo é implementado o algoritmo genético com a funções criadas acima. O algoritmo segue os passos abaixo:<h3>
<div>

(1) Gerada uma população alaeatória

(2) Calculado o fitness de todos os cromossomos

(3) Verificado se há o fitness máximo (solução)

(4) Seleciona os cromossomos mais promissores (fitness maior) para a nova população

(5) Seleciona, através de torneio, os cromossomos

(6) Cruza os cromossomos selecionados e cria a nova população (muda de geração)

(7) Refaz os passos 2, 3, 4, 5 e 6 até encontrar a solução ou atingir o número máximo de gerações
<div>

In [44]:
#Parametros
num_queen = 8
pop_max = 500
pop_inicial = pop_aletoria()
ger_atual = 0
ger_max = 1000
encontrou = False

#Enquanto não encontrar ou estiver na geração máxima
while(ger_atual != ger_max and encontrou != True):
    #população nova
    nova_pop = [[0 for i in range(num_queen)] for i in range(pop_max)]
    #fitness dos cromossomos
    fitnesses = all_fitness(pop_inicial)
    for i in range(0,len(fitnesses)):
        #caso encontre a solução
        if fitnesses[i]==28:
            encontrou = True
            solucao = pop_inicial[i]
            ger_solucao = ger_atual
            print("A solução foi encontrada:",solucao,"na geração ",ger_atual)
            break
        elif fitnesses[i]==27:
            solucao = pop_inicial[i]
    #elitismo
    elite = elitismo(fitnesses.copy())
    nova_pop[0] = pop_inicial[elite[0]]
    nova_pop[1] = pop_inicial[elite[1]]
    num_filhos = 2
    #Cruzamento
    while num_filhos < pop_max:
        ind_vencedores = selecao_torneio(fitnesses)
        filhos = crossover(pop_inicial[ind_vencedores[0]],pop_inicial[ind_vencedores[1]])
        filhos = mutacao(filhos,0.5)
        nova_pop[num_filhos] = filhos[0]
        nova_pop[num_filhos+1] = filhos[1]
        num_filhos = num_filhos + 2
    pop_inicial = nova_pop.copy()
    nova_pop.clear()
    ger_atual = ger_atual + 1
if(ger_atual==ger_max):
    print("A solução ótima não foi encontrada,a melhor solução encontrada foi: ",solucao)


A solução foi encontrada: [6, 2, 0, 5, 7, 4, 1, 3] na geração  13


<h3> Referências: <h3>
<h4> https://colab.research.google.com/drive/1liCLaczXpSgoYrw8UKcAaLvvBBeLDHDu?usp=sharing<h4>
<h4> https://www.kaggle.com/code/aminizahra/genetic-algorithm-8-queens-problem/notebook<h4>
<h4> https://github.com/mahdihassanzade/N-Queen-Problem-using-Genetic-Algorithm<h4>